In [1]:
import sys
sys.version

'3.7.2 (default, Dec 25 2018, 03:50:46) \n[GCC 7.3.0]'

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import os.path


In [3]:
from importlib import reload

In [4]:
import libmotivation
reload(libmotivation)
from libmotivation import *
import mkdata
reload(mkdata)
from mkdata import *

In [ ]:
user_list = get_users_with_choose_difficulty()

In [ ]:
user_to_difficulty_items = {}
for user in user_list:
  user_to_difficulty_items[user] = get_choose_difficulty_items_for_user(user)

In [ ]:
import arrow
from dateutil import tz

#timezone_list = list(timezone_set)
#print(timezone_list)
#print(list(map(to_hours_and_minutes, timezone_list)))

def parse_timezone_offset(datestr):
  date_parts = datestr.split(' ')
  for x in date_parts:
    if 'GMT' in x:
      output = (x.replace('GMT', ''))
      #timezone_set.add(output)
      return output
  raise Exception(datestr)

def to_hours_and_minutes(offset):
  if len(offset) != 5:
    raise Exception(offset)
  sign = offset[0]
  hours = int(offset[1:3])
  minutes = int(offset[3:])
  if sign == '+':
    return hours,minutes
  if sign == '-':
    return -hours,-minutes
  raise Exception(offset)

def adjust_timestamp_to_timezone_offset(timestamp, offset):
  hours,minutes = to_hours_and_minutes(offset)
  #print(hours, minutes)
  ar = arrow.get(timestamp / 1000.0)
  return ar.shift(hours=hours, minutes=minutes)

def get_time_adjusted_for_timezone(timestamp, datestr):
  offset = parse_timezone_offset(datestr)
  return adjust_timestamp_to_timezone_offset(timestamp, offset)

#print(adjust_timestamp_to_timezone(1548789649098.0,'-0600')) # 'Tue Jan 29 2019 13:20:49 GMT-0600 (Central Standard Time)',
print(get_time_adjusted_for_timezone(1548789649098.0, 'Tue Jan 29 2019 13:20:49 GMT-0600 (Central Standard Time)'))

In [ ]:


timezone_set = set()



timestamp_server = 1544629175088.0
timestamp_local = 1544629174633.0
#(timestamp_local - timestamp_server) / (1000*3600)
#arrow.get(timestamp_local / 1000)
#timestamp
#arrow.get('Wed Dec 12 2018 07:39:34 GMT-0800 (Pacific Standard Time)')
print(parse_timezone_offset('Wed Dec 12 2018 07:39:34 GMT-0800 (Pacific Standard Time)'))

In [ ]:
for user in user_list:
  difficulty_items = user_to_difficulty_items[user]
  for x in difficulty_items:
    parse_timezone_offset(x['localtime'])

In [ ]:
for user in user_list[1000:]:
  difficulty_items = user_to_difficulty_items[user]
  if len(difficulty_items) > 0:
    print(difficulty_items[0])
    break


In [ ]:
print(get_most_common_difficulty_for_user('8d2c9eb27dee2dc85bca705b'))

In [ ]:
print(get_most_common_difficulty_overall())

In [ ]:
def get_correctness_of_naive_overall_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_overall()
  if most_common == None:
    return None
  output['total'] = sum(difficulty_counts.values())
  if output['total'] == 0:
    return None
  output['correct'] = difficulty_counts.get(most_common, 0)
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [ ]:
def get_correctness_of_naive_peruser_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_for_user(user)
  if most_common == None:
    return None
  output['correct'] = difficulty_counts[most_common]
  output['total'] = sum(difficulty_counts.values())
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [ ]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_peruser_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

In [ ]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_overall_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

In [ ]:
num_correct_unnormalized = 0
num_total_unnormalized = 0
for user in user_list:
  correctness_info = get_correctness_of_naive_overall_strategy(user)
  if correctness_info == None:
    continue
  num_correct_unnormalized += correctness_info['correct']
  num_total_unnormalized += correctness_info['total']
  #accuracy = correctness_info['accuracy']
  #correctnes_of_naive_per_user.append(accuracy)

#print(np.median(correctnes_of_naive_per_user))
#print(np.mean(correctnes_of_naive_per_user_unnormalized))
print(num_correct_unnormalized / num_total_unnormalized)

In [ ]:
num_correct_unnormalized = 0
num_total_unnormalized = 0
for user in user_list:
  correctness_info = get_correctness_of_naive_peruser_strategy(user)
  if correctness_info == None:
    continue
  num_correct_unnormalized += correctness_info['correct']
  num_total_unnormalized += correctness_info['total']
  #accuracy = correctness_info['accuracy']
  #correctnes_of_naive_per_user.append(accuracy)

#print(np.median(correctnes_of_naive_per_user))
#print(np.mean(correctnes_of_naive_per_user_unnormalized))
print(num_correct_unnormalized / num_total_unnormalized)

In [ ]:
#n_features = 4 # the choice at each step. nothing, easy, medium, hard
#n_features = 5
n_features = 4 + 4 + 7 + 5 + 69 + 5 + get_num_languages()
n_categories = 4 # the number of categories we are categorizing among. nothing, easy, medium, hard


In [ ]:
difficulty_to_idx = {
  'nothing': 0,
  'easy': 1,
  'medium': 2,
  'hard': 3,
}

# def difficulty_to_idx(difficulty):
#   return {
#     'nothing': 0,
#     'easy': 1,
#     'medium': 2,
#     'hard': 3,
#   }[difficulty]

In [ ]:
import json

domain_to_productivity = json.load(open('domain_to_productivity.json'))
domain_to_category = json.load(open('domain_to_category.json'))

category_list = list(set(domain_to_category.values()))

domain_to_productivity_idx = {}
domain_to_category_idx = {}
for domain,productivity in domain_to_productivity.items():
  # should we do something about domains where productivity = 0?
  domain_to_productivity_idx[domain] = productivity + 2
domain_to_category_idx = {}
for domain,category in domain_to_category.items():
  # should we do something about domains where category = ''?
  category_idx = category_list.index(category)
  domain_to_category_idx[domain] = category_idx
print(len(set(domain_to_category_idx.values())))
print(len(set(domain_to_productivity_idx.values())))

In [ ]:
def url_to_domain(url):
  #url='https://www.hello.org/bye/'
  domain=url.split('//')[-1].split('/')[0]
  #print (domain)
  return domain

print(url_to_domain('https://www.hello.org/bye/'))

In [ ]:


def make_tensors_for_user(user):
  features = make_features_for_user(user)
  return make_tensors_from_features(features)

# def make_tensor_from_prior_difficulties_list(prior_difficulties):
#   tensor = torch.zeros(len(prior_difficulties), 1, n_features)
#   for idx,difficulty in enumerate(prior_difficulties):
#     difficulty_idx = difficulty_to_idx[difficulty]
#     tensor[idx][0][difficulty_idx] = 1
#   return tensor

def make_tensor_from_chosen_difficulty(chosen_difficulty):
  difficulty_idx = difficulty_to_idx[chosen_difficulty]
  tensor = torch.tensor([difficulty_idx], dtype=torch.long)
  return tensor

def make_tensors_from_features_v1(features):
  output = []
  for feature in features:
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 4 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
    #feature_tensor = make_tensor_from_prior_difficulties_list(prior_difficulties)
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v2(features):
  output = []
  for feature in features:
    hour = feature['arrow_time'].hour
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 1 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][0] = difficulty_idx
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def hour_to_hour_idx_4cat(hour): # hour: 0 to 24
  if 0 <= hour <= 6:
    return 0
  if 6 < hour <= 12:
    return 1
  if 12 < hour <= 18:
    return 2
  if 18 < hour <= 24:
    return 3
  raise Exception(hour)

def make_tensors_from_features_v3(features):
  output = []
  for feature in features:
    hour = feature['arrow_time'].hour
    hour_idx = hour_to_hour_idx_4cat(hour)
    weekday_idx = feature['arrow_time'].weekday()
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 15 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][8 + weekday_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v4(features):
  output = []
  for feature in features:
    url = feature['url']
    domain = url_to_domain(url)
    have_productivity_idx = False
    if domain in domain_to_productivity_idx:
      domain_productivity_idx = domain_to_productivity_idx[domain]
      have_productivity_idx = True
    else:
      domain_productivity_idx = None
    have_category_idx = False
    if domain in domain_to_category_idx:
      domain_category_idx = domain_to_productivity_idx[domain]
      have_category_idx = True
    else:
      domain_category_idx = None
    hour = feature['arrow_time'].hour
    hour_idx = hour_to_hour_idx_4cat(hour)
    weekday_idx = feature['arrow_time'].weekday()
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties), 1, n_features) # n_features = 15 in this version
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      feature_tensor[idx][0][difficulty_idx] = 1
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def get_domain_features_from_url(url):
  domain = url_to_domain(url)
  have_productivity_idx = False
  if domain in domain_to_productivity_idx:
    domain_productivity_idx = domain_to_productivity_idx[domain]
    have_productivity_idx = True
  else:
    domain_productivity_idx = None
  have_category_idx = False
  if domain in domain_to_category_idx:
    domain_category_idx = domain_to_productivity_idx[domain]
    have_category_idx = True
  else:
    domain_category_idx = None
  return domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx

def get_time_features_from_arrow(arrow_time):
  hour = arrow_time.hour
  hour_idx = hour_to_hour_idx_4cat(hour)
  weekday_idx = arrow_time.weekday()
  return hour_idx,weekday_idx

def make_tensors_from_features_v5(features): # this one cheats because we have the reference information included
  output = []
  for feature in features:
    url = feature['url']
    domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(url)
    hour_idx,weekday_idx = get_time_features_from_arrow(feature['arrow_time'])
    chosen_difficulty = feature['difficulty']
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties)+1, 1, n_features) # n_features = 15 in this version
    # features for current timestep
    idx = len(prior_difficulties)
    difficulty = feature['difficulty']
    difficulty_idx = difficulty_to_idx[difficulty]
    #if len(prior_difficulties) > 0:
    #  feature_tensor[idx][0][difficulty_to_idx[prior_difficulties[-1]]] = 1
    #feature_tensor[idx][0][difficulty_idx] = 1 # this is an impossible feature. see whether it learns corectly
    feature_tensor[idx][0][4 + hour_idx] = 1
    feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
    if have_productivity_idx:
      feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
    if have_category_idx:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    # features for previous timesteps
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      #feature_tensor[idx][0][difficulty_idx] = 1
      hour_idx,weekday_idx = get_time_features_from_arrow(feature['prior_arrow_times'][idx])
      domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(feature['prior_urls'][idx])
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

def make_tensors_from_features_v6(features): # this one cheats because we have the reference information included
  output = []
  for feature in features:
    url = feature['url']
    domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(url)
    hour_idx,weekday_idx = get_time_features_from_arrow(feature['arrow_time'])
    chosen_difficulty = feature['difficulty']
    language_indexes = convert_language_list_to_language_indexes(feature['languages'])
    prior_difficulties = feature['prior_difficulties']
    user = feature['user']
    category_tensor = make_tensor_from_chosen_difficulty(chosen_difficulty)
    feature_tensor = torch.zeros(len(prior_difficulties)+1, 1, n_features) # n_features = 15 in this version
    # features for current timestep
    idx = len(prior_difficulties)
    difficulty = feature['difficulty']
    difficulty_idx = difficulty_to_idx[difficulty]
    #if len(prior_difficulties) > 0:
    #  feature_tensor[idx][0][difficulty_to_idx[prior_difficulties[-1]]] = 1
    #feature_tensor[idx][0][difficulty_idx] = 1 # this is an impossible feature. see whether it learns corectly
    feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + difficulty_to_idx[feature['initial_difficulty']]] = 1
    for language_index in language_indexes:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + 5 + language_index] = 1
    feature_tensor[idx][0][4 + hour_idx] = 1
    feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
    if have_productivity_idx:
      feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
    if have_category_idx:
      feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
    # features for previous timesteps
    for idx,difficulty in enumerate(prior_difficulties):
      difficulty_idx = difficulty_to_idx[difficulty]
      #feature_tensor[idx][0][difficulty_idx] = 1
      hour_idx,weekday_idx = get_time_features_from_arrow(feature['prior_arrow_times'][idx])
      domain_productivity_idx,have_productivity_idx,domain_category_idx,have_category_idx = get_domain_features_from_url(feature['prior_urls'][idx])
      feature_tensor[idx][0][4 + hour_idx] = 1
      feature_tensor[idx][0][4 + 4 + weekday_idx] = 1
      if have_productivity_idx:
        feature_tensor[idx][0][4 + 4 + 7 + domain_productivity_idx] = 1
      if have_category_idx:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + domain_category_idx] = 1
      feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + difficulty_to_idx[feature['initial_difficulty']]] = 1
      for language_index in language_indexes:
        feature_tensor[idx][0][4 + 4 + 7 + 5 + 69 + 5 + language_index] = 1
    output.append({'user': user, 'chosen_difficulty': chosen_difficulty, 'category': category_tensor, 'feature': feature_tensor})
  return output

make_tensors_from_features = make_tensors_from_features_v6

history_length = 10

def make_features_for_user(user):
  output = []
  difficulty_items = get_choose_difficulty_items_for_user(user)
  prior_difficulties = []
  prior_urls = []
  prior_arrow_times = []
  languages = get_languages_for_user(user)
  initial_difficulty = get_initial_difficulty_for_user(user)
  if initial_difficulty == None:
    return []
  for item in difficulty_items:
    if 'is_random' in item and item['is_random'] == True:
      continue
    if 'type' not in item:
      continue
    if item['type'] != 'action':
      continue
    if 'difficulty' not in item:
      continue
    difficulty = item['difficulty']
    url = item['url']
    arrow_time = get_time_adjusted_for_timezone(item['timestamp_local'], item['localtime'])
    output.append({'url': url, 'user': user, 'initial_difficulty': initial_difficulty, 'languages': languages, 'difficulty': difficulty, 'arrow_time': arrow_time, 'prior_urls': prior_urls[:], 'prior_difficulties': prior_difficulties[:], 'prior_arrow_times': prior_arrow_times[:]})
    prior_difficulties.append(difficulty)
    prior_urls.append(url)
    prior_arrow_times.append(arrow_time)
    if len(prior_difficulties) > history_length:
      prior_difficulties = prior_difficulties[-history_length:]
      prior_urls = prior_urls[-history_length:]
      prior_arrow_times  = prior_arrow_times[-history_length:]
  return output


In [ ]:
print(arrow.get().shift(hours=-8, days=-4).weekday())

In [ ]:
#print(make_tensors_from_features([{'url': 'facebook.com', 'user': 'foobar', 'difficulty': 'easy', 'prior_difficulties': [], 'arrow_time': arrow.get().shift(hours=-8)}]))
#print(make_tensors_from_features([{'url': 'facebook.com', 'user': 'foobar', 'difficulty': 'easy', 'prior_difficulties': ['easy', 'nothing'], 'arrow_time': arrow.get().shift(hours=-8)}]))


In [ ]:
all_data = []
for user in user_list:
  print(user)
  tensors = make_tensors_for_user(user)
  for tensor in tensors:
    all_data.append(tensor)

In [ ]:
def tensor_to_difficulty(tensor):
  difficulty_idx = tensor[0].data.cpu().numpy()
  return ['nothing', 'easy', 'medium', 'hard'][difficulty_idx]

print(make_tensor_from_chosen_difficulty('hard'))
print(tensor_to_difficulty((make_tensor_from_chosen_difficulty('hard'))))

In [ ]:
training_data = all_data[:int(math.floor(len(all_data)*9/10))]
dev_and_test_data = all_data[len(training_data):]
dev_data = dev_and_test_data[:int(math.floor(len(dev_and_test_data)/2))]
test_data = dev_and_test_data[len(dev_data):]

def iterateTrainingData():
  output = []
  for data in training_data:
    category = tensor_to_difficulty(data['category'])
    #yield category,data['category'],data['feature']
    output.append((category,data['category'],data['feature']))
  np.random.shuffle(output)
  return output

def randomTrainingExample():
  data = random.choice(training_data)
  category = tensor_to_difficulty(data['category'])
  return category,data['category'],data['feature']

#def makeDataTensor(lines_of_data):
#  output = 

In [ ]:
#print(randomTrainingExample())
category,category_tensor,line_tensor = iterateTrainingData()[4]
print(line_tensor.size())
#print('line tensor size dimension 1 is')
#print(line_tensor.size()[0])
#train(category_tensor.cuda(), line_tensor.cuda())


In [ ]:
criterion = nn.NLLLoss()


In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden().cuda()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [ ]:
torch.cuda.device_count()

In [ ]:

class RNN1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN1, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128

class RNN2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN2, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        print('pre softmax output size is')
        print(output.size())
        output = self.softmax(output)
        print('post softmax output size is')
        print(output.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 256

rnn = RNN2(n_features, n_hidden, n_categories).cuda()

class RNN3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(RNN3, self).__init__()

        self.drop = nn.Dropout(dropout)
        self.nlayers = num_layers
        self.nhid = hidden_size
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        #self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        #self.i2o = nn.Linear(input_size + hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
      #print('input.shape in forward is')
      #print(input.size())
      #print('hidden[0].shape in forward is')
      #print(hidden[0].size())
      #print('hidden[1].shape in forward is')
      #print(hidden[1].size())
      input = input.view((input.size(0), 1, input.size(1)))
      output,hidden = self.rnn(input, hidden)
      #print('pre softmax output size is')
      #print(output.size())
      output = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
      output = self.softmax(output)
      #print('post softmax output size is')
      #print(output.size())
      return output, hidden

    def initHidden(self, bsz=1):
        return (torch.zeros(2, 1, 256).cuda(), torch.zeros(2, 1, 256).cuda())
        #weight = next(self.parameters())
        #return (weight.new_zeros(self.nlayers, bsz, self.nhid),
        #       weight.new_zeros(self.nlayers, bsz, self.nhid))
    
    #def initHidden(self):
    #    return torch.zeros(10, 2, 1, self.hidden_size)

n_hidden = 256

rnn3 = RNN3(n_features, n_hidden, n_categories).cuda()

In [ ]:
# model = model.RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout, args.tied).to(device)
# LSTM 33278 200 200 2 0.2 False
# rnn_type = args.model='LSTM'
# ntoken = ntokens=33278 (number of tokens. presumably the number of words in the english language)
# ninp = args.emsize=200 (size of word embeddings)
# nhid = args.nhid=200 (number of hidden units per layer)
# nlayers = args.nlayers=2 (number of layers)
# dropout = args.dropout=0.2 (dropout applied to layers (0 = no dropout))
# tie_weights = args.tied=False (tie the word embedding and softmax weights)


import torch.nn as nn

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        #print('input is')
        #print(input.size()) # torch.Size([35, 20]) # 35 = sequence length, 20 = batch size
        #emb = self.drop(self.encoder(input))
        #print('emb is')
        #print(emb.size()) # torch.Size([35, 20, 200]) # 35 = sequence length, 20 = batch size, 200 = embedding size
        emb = input
        output, hidden = self.rnn(emb, hidden)
        return output,hidden
        #output = self.drop(output)
        #decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        #return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        return (weight.new_zeros(self.nlayers, bsz, self.nhid), weight.new_zeros(self.nlayers, bsz, self.nhid))

rnn2 = RNNModel(rnn_type='LSTM', ntoken=4, ninp=4, nhid=200, nlayers=2, dropout=0, tie_weights=False)

# def makeIntoTensor(data_list):
#   output = {}
#   line_to_user = {}
  
#   for idx,item in enumerate(training_data):
#     print(idx)
#     print(item)
#     print(item.size())
#     break

# def makeTrainingData():
#   return makeIntoTensor(training_data)
  

# makeTrainingData() 

In [ ]:
def save_model(rnn, criterion, epoch, loss, filename):
  torch.save({
    'epoch': epoch,
    'model_state_dict': rnn.state_dict(),
    'optimizer_state_dict': criterion.state_dict(),
    'loss': loss,
  }, filename)

#print(criterion.state_dict())

In [ ]:
def categoryFromOutput(output):
  top_n,top_i = output.topk(1)
  category_i = top_i[0].item()
  return ['nothing','easy','medium','hard'][category_i],category_i
#print(output.topk(1))
#print(categoryFromOutput(output))

In [ ]:
rnn = RNN3(n_features, n_hidden, n_categories, dropout=0.2).cuda()

#learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train2(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    hidden = repackage_hidden(hidden)
    #hidden = hidden.cuda()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

n_iters = 500
print_every = 1
plot_every = 1
all_losses = []

rnn.train()
all_training_items = iterateTrainingData()
for epoch in range(1, n_iters + 1):
  print('iteration', epoch)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = train2(category_tensor.cuda(), line_tensor.cuda())
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(rnn, criterion, epoch, current_loss, 'model_rnn_v9_epoch' + str(epoch) + '.pt')

In [ ]:
rnn.train()
#all_training_items = iterateTrainingData()
for iter in range(1, n_iters + 1):
  print('iteration', iter)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = train2(category_tensor.cuda(), line_tensor.cuda())
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(iter, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)

In [ ]:
import time
import math
import random

n_iters = 100
print_every = 1
plot_every = 1



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
  #category,category_tensor, line_tensor = randomTrainingExample()
  #if True:
  all_training_items = iterateTrainingData()
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    if line_tensor.size()[0] == 0:
      continue
    output, loss = train(category_tensor.cuda(), line_tensor.cuda())
    current_loss += loss
    if idx % 1000 == 0:
      print(iter, ':', idx, '/', len(all_training_items))

  # Print iter number, loss, name and guess
  if iter % print_every == 0:
      guess, guess_i = categoryFromOutput(output)
      correct = '✓' if guess == category else '✗ (%s)' % category
      print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

  # Add current loss avg to list of losses
  if iter % plot_every == 0:
      all_losses.append(current_loss / plot_every)
      current_loss = 0

# for iter in range(1, n_iters + 1):
#     category,category_tensor, line_tensor = randomTrainingExample()
#     if line_tensor.size()[0] == 0:
#       continue
#     output, loss = train(category_tensor.cuda(), line_tensor.cuda())
#     current_loss += loss

#     # Print iter number, loss, name and guess
#     if iter % print_every == 0:
#         guess, guess_i = categoryFromOutput(output)
#         correct = '✓' if guess == category else '✗ (%s)' % category
#         print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

#     # Add current loss avg to list of losses
#     if iter % plot_every == 0:
#         all_losses.append(current_loss / plot_every)
#         current_loss = 0

In [ ]:
%matplotlib inline


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
#plt.plot(all_losses[:10])
plt.plot(all_losses)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
#plt.plot(all_losses[:10])
plt.plot(all_losses)

In [ ]:
all_categories = ['nothing', 'easy', 'medium', 'hard']

In [ ]:
# Just return an output given a line
def evaluate3(model, line_tensor):
    model.eval()
    hidden = model.initHidden() #.cuda()

    for i in range(line_tensor.size()[0]):
        output, hidden = model(line_tensor[i], hidden)
    
    return output

In [ ]:
# Just return an output given a line
def evaluate(line_tensor):
    rnn.eval()
    hidden = rnn.initHidden() #.cuda()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    return output

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000



# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, category_tensor, line_tensor = randomTrainingExample()
    if line_tensor.size()[0] == 0:
      continue
    output = evaluate(line_tensor.cuda())
    guess, guess_i = categoryFromOutput(output)
    category_i = difficulty_to_idx[category]
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
def evaluate_model_on_dataset(model, dataset):
  num_correct = 0
  num_incorrect = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = difficulty_to_idx[difficulty]
    predicted_tensor = evaluate3(model, feature_tensor.cuda())
    predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
    if predicted_difficulty_idx == difficulty_idx:
      num_correct += 1
    else:
      num_incorrect += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
    #print(predicted_difficulty)
    #print(difficulty)
    #break

  print('num correct: ' + str(num_correct))
  print('num incorrect: ' + str(num_incorrect))
  print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in training_data:
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_tensor = evaluate(feature_tensor.cuda())
  predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_rnn_v9_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  #model = SelfAttentionLSTM()
  model = RNN3(n_features, n_hidden, n_categories, dropout=0.2).cuda()
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, dev_data)
  #break


In [ ]:
epoch = 0
while True:
  epoch += 1
  model_filename = 'model_rnn_v9_epoch' + str(epoch) + '.pt'
  if not os.path.isfile(model_filename):
    break
  print(model_filename)
  #model = SelfAttentionLSTM()
  model = RNN3(n_features, n_hidden, n_categories, dropout=0.2).cuda()
  model_state = torch.load(model_filename)
  model.load_state_dict(model_state['model_state_dict'])
  model.eval()
  evaluate_model_on_dataset(model, training_data)
  #break


In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_tensor = evaluate(feature_tensor.cuda())
  predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_difficulty = get_most_common_difficulty_for_user(user)
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
def get_most_recent_difficulty(tensor):
  difficulty_arr = list(tensor[-1][0].data.cpu().numpy())
  max_idx = difficulty_arr.index(1)
  print(len(difficulty_arr))
  print(max_idx)
  return ['nothing', 'easy', 'medium', 'hard'][max_idx]

#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  predicted_difficulty = get_most_recent_difficulty(feature_tensor)
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))

In [ ]:
#confusion = torch.zeros(n_categories, n_categories)
num_correct = 0
num_incorrect = 0
confusion = torch.zeros(n_categories, n_categories)

for item in dev_data:
  user = item['user']
  category_tensor = item['category']
  feature_tensor = item['feature']
  if feature_tensor.size()[0] == 0:
      continue
  difficulty = tensor_to_difficulty(category_tensor)
  difficulty_idx = difficulty_to_idx[difficulty]
  #predicted_difficulty = get_most_common_difficulty_for_user(user)
  predicted_difficulty = get_most_common_difficulty_overall()
  predicted_difficulty_idx = difficulty_to_idx[predicted_difficulty]
  predicted_tensor = make_tensor_from_chosen_difficulty(predicted_difficulty)
  #predicted_tensor = evaluate(feature_tensor.cuda())
  #predicted_difficulty,predicted_difficulty_idx = categoryFromOutput(predicted_tensor)
  if predicted_difficulty_idx == difficulty_idx:
    num_correct += 1
  else:
    num_incorrect += 1
  confusion[difficulty_idx][predicted_difficulty_idx] += 1
  #print(predicted_difficulty)
  #print(difficulty)
  #break

print('num correct: ' + str(num_correct))
print('num incorrect: ' + str(num_incorrect))
print('percentage correct: ' + str(num_correct / (num_correct + num_incorrect)))